In [ ]:
#Modelo regressao com infos:

# ============ IMPORTAÇÕES COMPLETAS ============
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import box, Polygon
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, \
    recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from scipy import ndimage
import seaborn as sns
import time
import warnings
import xgboost as xgb

warnings.filterwarnings('ignore')


# ============ CONFIGURAÇÃO ============
class Config:
    """Configurações para regressão preditiva temporal"""
    DATA_PATH = Path("C:/Users/Rikar/Downloads/Projeto/csv/csv/")
    LAT_COL, LON_COL = "LATITUDE", "LONGITUDE"
    CELL_SIZE = 0.01

    # 🔥 DEFINIÇÃO EXPLÍCITA TEMPORAL
    ANOS_DISPONIVEIS = [2019, 2020, 2021, 2022]
    ANOS_TREINO = [2019, 2020, 2021]
    ANO_TESTE = 2022

    # Garantir que não há overlap
    assert ANO_TESTE not in ANOS_TREINO, "❌ Ano de teste não pode estar no treino!"
    assert ANO_TESTE in ANOS_DISPONIVEIS, "❌ Ano de teste deve estar disponível!"

    # Parâmetros XGBoost
    XGB_PARAMS = {
        'n_estimators': 300,
        'max_depth': 10,
        'learning_rate': 0.05,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1,
        'random_state': 42,
        'n_jobs': -1
    }

    HOTSPOT_PERCENTIL = 85
    THRESHOLD_MINIMO = 5  # Mínimo de crimes para ser considerado hotspot


# ============ CARREGAMENTO DE DADOS ============
class DataLoader:
    def __init__(self, config):
        self.config = config

    def carregar_ano(self, ano):
        """Carrega dados de um ano específico"""
        try:
            arquivo = self.config.DATA_PATH / f"SPSafe_{ano}.csv"
            if not arquivo.exists():
                print(f"   ❌ Arquivo {arquivo} não encontrado")
                return None

            df = pd.read_csv(
                arquivo,
                sep=None,
                engine="python",
                encoding_errors="ignore",
                usecols=[self.config.LAT_COL, self.config.LON_COL, "NATUREZA_APURADA",
                         "DATA_OCORRENCIA", "HORA_OCORRENCIA", "PERIODO_OCORRENCIA",
                         "TIPO_LOCAL", "IDADE_PESSOA"],
                dtype={
                    'NATUREZA_APURADA': 'category',
                    'PERIODO_OCORRENCIA': 'category',
                    'TIPO_LOCAL': 'category'
                }
            )
            df['ANO'] = ano
            print(f"   ✅ {ano}: {len(df):,} registros")
            return df
        except Exception as e:
            print(f"   ❌ Erro ao carregar {ano}: {e}")
            return None


# ============ PROCESSAMENTO ESPACIAL ============
class SpatialProcessor:
    def __init__(self, config):
        self.config = config

    def processar_dados(self, df, nome_conjunto):
        """Processa e limpa os dados"""
        print(f"🔧 Processando {nome_conjunto} ({len(df):,} registros)...")

        # Limpeza básica
        initial_count = len(df)
        df = df.dropna(subset=[self.config.LAT_COL, self.config.LON_COL])
        df = df[(df[self.config.LAT_COL].between(-90, 90)) &
                (df[self.config.LON_COL].between(-180, 180))]

        print(f"   ✅ {len(df):,} coordenadas válidas (de {initial_count:,} inicial)")

        # Features temporais
        df['DATA_OCORRENCIA'] = pd.to_datetime(df['DATA_OCORRENCIA'], errors='coerce')
        df['MES'] = df['DATA_OCORRENCIA'].dt.month
        df['DIA_SEMANA'] = df['DATA_OCORRENCIA'].dt.dayofweek
        df['FIM_DE_SEMANA'] = df['DIA_SEMANA'].isin([5, 6]).astype(int)

        # Criar geodataframe
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df[self.config.LON_COL], df[self.config.LAT_COL]),
            crs="EPSG:4326"
        )
        return gdf

    def carregar_limites_sp(self):
        """Carrega os limites de São Paulo"""
        print("🏞️ Carregando limites do Estado de São Paulo...")
        try:
            fonte_alternativa = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
            estados = gpd.read_file(fonte_alternativa)
            sao_paulo = estados[estados['name'] == 'São Paulo']
            if not sao_paulo.empty:
                print("✅ Limites carregados da fonte alternativa!")
                return sao_paulo.reset_index(drop=True)
            else:
                raise ValueError("São Paulo não encontrado")
        except Exception as e:
            print(f"❌ Erro: {e}")
            # Fallback: bounding box de SP
            sp_bbox = Polygon([
                (-53.11, -25.33), (-44.15, -25.33),
                (-44.15, -19.77), (-53.11, -19.77),
                (-53.11, -25.33)
            ])
            sao_paulo_fallback = gpd.GeoDataFrame({
                'nome': ['São Paulo'], 'sigla': ['SP'], 'geometry': [sp_bbox]
            }, crs="EPSG:4326")
            print("✅ Usando bounding box como fallback")
            return sao_paulo_fallback

    def criar_grid_e_features(self, gdf, sao_paulo, nome_conjunto):
        """Cria grid espacial e calcula features"""
        print(f"📦 Criando grid e features para {nome_conjunto}...")

        # Filtrar dados para SP
        gdf_sp = gpd.sjoin(gdf, sao_paulo, how="inner", predicate="within")
        print(f"   ✅ {len(gdf_sp):,} ocorrências dentro de São Paulo")

        # Criar grid
        xmin, ymin, xmax, ymax = sao_paulo.total_bounds
        xmin = np.floor(xmin / self.config.CELL_SIZE) * self.config.CELL_SIZE
        ymin = np.floor(ymin / self.config.CELL_SIZE) * self.config.CELL_SIZE
        xmax = np.ceil(xmax / self.config.CELL_SIZE) * self.config.CELL_SIZE
        ymax = np.ceil(ymax / self.config.CELL_SIZE) * self.config.CELL_SIZE

        x_coords = np.arange(xmin, xmax, self.config.CELL_SIZE)
        y_coords = np.arange(ymin, ymax, self.config.CELL_SIZE)

        # Criar células do grid
        polygons = []
        for x in x_coords:
            for y in y_coords:
                polygons.append(box(x, y, x + self.config.CELL_SIZE, y + self.config.CELL_SIZE))

        grid_sp = gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326")
        grid_sp["cell_id"] = grid_sp.index

        # Filtrar células dentro de SP
        grid_sp_filtered = gpd.sjoin(grid_sp, sao_paulo, how="inner", predicate="intersects")

        # Spatial join seguro
        gdf_sp_clean = gdf_sp.reset_index(drop=True).copy()
        grid_sp_clean = grid_sp_filtered.reset_index(drop=True).copy()

        for df_temp in [gdf_sp_clean, grid_sp_clean]:
            if 'index_right' in df_temp.columns:
                df_temp.drop(columns=['index_right'], inplace=True)
            if 'index_left' in df_temp.columns:
                df_temp.drop(columns=['index_left'], inplace=True)

        joined_sp = gpd.sjoin(
            gdf_sp_clean,
            grid_sp_clean[['geometry', 'cell_id']],
            how="left",
            predicate="within",
            lsuffix='_point',
            rsuffix='_grid'
        )

        # Contagem por célula
        cell_counts_sp = joined_sp["cell_id"].value_counts().rename_axis("cell_id").reset_index(name="count")
        grid_final = grid_sp_clean.merge(cell_counts_sp, on="cell_id", how="left").fillna({"count": 0})

        # Converter coordenadas
        grid_final_proj = grid_final.to_crs('EPSG:3857')
        grid_final_proj['centroid'] = grid_final_proj.geometry.centroid
        grid_final['x'] = grid_final_proj.centroid.x
        grid_final['y'] = grid_final_proj.centroid.y

        # Features avançadas
        features_df = self._extrair_features_avancadas(joined_sp, grid_final)
        grid_final = grid_final.merge(features_df, on='cell_id', how='left')

        # Preencher valores faltantes
        feature_cols = ['periodos_diversidade', 'naturezas_diversidade', 'locais_diversidade',
                        'idade_media', 'idade_std', 'concentracao_temporal', 'densidade_vizinhanca']
        for col in feature_cols:
            if col in grid_final.columns:
                grid_final[col] = grid_final[col].fillna(0)
            else:
                grid_final[col] = 0

        print(f"   ✅ {len(grid_final):,} células processadas")
        return grid_final

    def _extrair_features_avancadas(self, joined_sp, grid_final):
        """Extrai features avançadas dos dados"""
        if len(joined_sp) == 0:
            return pd.DataFrame({
                'cell_id': grid_final['cell_id'],
                'periodos_diversidade': 0, 'naturezas_diversidade': 0,
                'locais_diversidade': 0, 'idade_media': 0, 'idade_std': 0,
                'concentracao_temporal': 0, 'densidade_vizinhanca': 0
            })

        try:
            grouped = joined_sp.groupby('cell_id')

            # Diversidade de características
            diversidade_periodos = grouped['PERIODO_OCORRENCIA'].nunique().rename('periodos_diversidade').reset_index()
            diversidade_naturezas = grouped['NATUREZA_APURADA'].nunique().rename('naturezas_diversidade').reset_index()
            diversidade_locais = grouped['TIPO_LOCAL'].nunique().rename('locais_diversidade').reset_index()

            # Estatísticas de idade
            if 'IDADE_PESSOA' in joined_sp.columns:
                estatisticas_idade = grouped['IDADE_PESSOA'].agg(['mean', 'std']).reset_index()
                estatisticas_idade = estatisticas_idade.rename(columns={'mean': 'idade_media', 'std': 'idade_std'})
            else:
                estatisticas_idade = pd.DataFrame({
                    'cell_id': grid_final['cell_id'],
                    'idade_media': 0, 'idade_std': 0
                })

            # Concentração temporal
            concentracao_temporal_df = self._calcular_concentracao_temporal(grouped, grid_final)

            # Densidade espacial
            densidade_df = self._calcular_densidade_espacial(grid_final)

            # Consolidar features
            features_df = diversidade_periodos.copy()
            features_df = features_df.merge(diversidade_naturezas, on='cell_id', how='left')
            features_df = features_df.merge(diversidade_locais, on='cell_id', how='left')
            features_df = features_df.merge(estatisticas_idade, on='cell_id', how='left')
            features_df = features_df.merge(concentracao_temporal_df, on='cell_id', how='left')
            features_df = features_df.merge(densidade_df, on='cell_id', how='left')

            return features_df.fillna(0)

        except Exception as e:
            print(f"   ⚠️  Erro ao extrair features: {e}")
            return pd.DataFrame({
                'cell_id': grid_final['cell_id'],
                'periodos_diversidade': 0, 'naturezas_diversidade': 0,
                'locais_diversidade': 0, 'idade_media': 0, 'idade_std': 0,
                'concentracao_temporal': 0, 'densidade_vizinhanca': 0
            })

    def _calcular_concentracao_temporal(self, grouped, grid_final):
        """Calcula concentração temporal dos crimes"""

        def calcular_concentracao(group_df):
            if len(group_df) == 0 or 'MES' not in group_df.columns:
                return 0
            mes_counts = group_df['MES'].value_counts()
            return mes_counts.max() / len(group_df)

        cells_com_dados = grid_final[grid_final['count'] > 0]['cell_id']
        concentracao_data = []

        for cell_id in cells_com_dados:
            try:
                group_data = grouped.get_group(cell_id)
                concentracao = calcular_concentracao(group_data)
                concentracao_data.append({'cell_id': cell_id, 'concentracao_temporal': concentracao})
            except KeyError:
                concentracao_data.append({'cell_id': cell_id, 'concentracao_temporal': 0})

        return pd.DataFrame(concentracao_data)

    def _calcular_densidade_espacial(self, grid_final):
        """Calcula densidade espacial usando suavização gaussiana"""
        try:
            x_coords_density = np.linspace(grid_final['x'].min(), grid_final['x'].max(), 50)
            y_coords_density = np.linspace(grid_final['y'].min(), grid_final['y'].max(), 50)

            density_matrix = np.zeros((len(y_coords_density), len(x_coords_density)))

            for idx, row in grid_final.iterrows():
                x_idx = np.abs(x_coords_density - row['x']).argmin()
                y_idx = np.abs(y_coords_density - row['y']).argmin()
                if x_idx < len(x_coords_density) and y_idx < len(y_coords_density):
                    density_matrix[y_idx, x_idx] = row['count']

            density_smoothed = ndimage.gaussian_filter(density_matrix, sigma=1)

            densidade_data = []
            for idx, row in grid_final.iterrows():
                x_idx = np.abs(x_coords_density - row['x']).argmin()
                y_idx = np.abs(y_coords_density - row['y']).argmin()
                if x_idx < len(x_coords_density) and y_idx < len(y_coords_density):
                    densidade = float(density_smoothed[y_idx, x_idx])
                else:
                    densidade = 0.0
                densidade_data.append({'cell_id': row['cell_id'], 'densidade_vizinhanca': densidade})

            return pd.DataFrame(densidade_data)
        except Exception as e:
            print(f"   ⚠️  Erro no cálculo de densidade: {e}")
            return pd.DataFrame({'cell_id': grid_final['cell_id'], 'densidade_vizinhanca': 0.0})


# ============ PROCESSADOR TEMPORAL SEGURO ============
class SafeTemporalProcessor:
    """Processamento que GARANTE separação temporal"""

    def __init__(self, config):
        self.config = config
        self.scaler = None
        self.features_para_usar = None

    def processar_conjunto_treino(self, anos_treino):
        """Processa APENAS dados de treino - sem informação do teste"""
        print("📊 Processando conjunto de TREINO...")

        grids_treino = []
        for ano in anos_treino:
            grid = self._processar_ano_individual(ano, f"Treino {ano}")
            if grid is not None:
                grids_treino.append(grid)

        if not grids_treino:
            raise ValueError("❌ Nenhum dado de treino processado!")

        # Combinar treino
        grid_treino_combinado = pd.concat(grids_treino, ignore_index=True)

        # Definir features APENAS com base no treino
        self.features_para_usar = self._definir_features(grid_treino_combinado)

        # Treinar scaler APENAS com dados de treino
        X_treino = grid_treino_combinado[self.features_para_usar]
        self.scaler = StandardScaler()
        X_treino_scaled = self.scaler.fit_transform(X_treino)

        y_treino = grid_treino_combinado['count']

        print(f"   ✅ Treino: {len(grid_treino_combinado):,} células")
        print(f"   ✅ Features: {len(self.features_para_usar)} variáveis")

        return X_treino_scaled, y_treino, grid_treino_combinado

    def processar_conjunto_teste(self, ano_teste):
        """Processa dados de teste usando APENAS informações do treino"""
        print(f"📊 Processando conjunto de TESTE ({ano_teste})...")

        grid_teste = self._processar_ano_individual(ano_teste, f"Teste {ano_teste}")
        if grid_teste is None:
            raise ValueError(f"❌ Não foi possível processar teste {ano_teste}")

        # Usar APENAS features definidas no treino
        features_disponiveis = [f for f in self.features_para_usar if f in grid_teste.columns]

        # Preencher features faltantes com 0
        for feature in self.features_para_usar:
            if feature not in grid_teste.columns:
                print(f"   ⚠️  Feature {feature} não encontrada no teste, preenchendo com 0")
                grid_teste[feature] = 0

        X_teste = grid_teste[self.features_para_usar]

        # Aplicar scaler treinado APENAS no treino
        if self.scaler is None:
            raise ValueError("❌ Scaler não foi treinado no conjunto de treino!")

        X_teste_scaled = self.scaler.transform(X_teste)
        y_teste = grid_teste['count']

        print(f"   ✅ Teste: {len(grid_teste):,} células")

        return X_teste_scaled, y_teste, grid_teste

    def _definir_features(self, grid_treino):
        """Define features APENAS com base nos dados de treino"""
        features_candidatas = ['x', 'y', 'periodos_diversidade', 'naturezas_diversidade',
                               'locais_diversidade', 'idade_media', 'idade_std',
                               'concentracao_temporal', 'densidade_vizinhanca']

        # Manter apenas features presentes no treino
        features_validas = [f for f in features_candidatas if f in grid_treino.columns]

        # Verificar variância (remover features constantes)
        features_finais = []
        for feature in features_validas:
            if grid_treino[feature].std() > 0.001:  # Threshold mínimo de variância
                features_finais.append(feature)
            else:
                print(f"   ⚠️  Removendo feature {feature} (variância muito baixa)")

        return features_finais

    def _processar_ano_individual(self, ano, nome):
        """Processa um ano individualmente"""
        print(f"   🔧 Processando {nome}...")

        try:
            # Carregar dados
            data_loader = DataLoader(self.config)
            df_ano = data_loader.carregar_ano(ano)
            if df_ano is None:
                return None

            # Processar
            spatial_processor = SpatialProcessor(self.config)
            gdf_ano = spatial_processor.processar_dados(df_ano, nome)
            sao_paulo = spatial_processor.carregar_limites_sp()
            grid_ano = spatial_processor.criar_grid_e_features(gdf_ano, sao_paulo, nome)

            return grid_ano

        except Exception as e:
            print(f"   ❌ Erro ao processar {nome}: {e}")
            return None


# ============ VISUALIZAÇÃO ============
class RegressionVisualizator:
    """Gerencia visualizações para regressão"""

    def __init__(self, config):
        self.config = config

    def criar_mapa_regressao(self, grid_teste, y_real, y_pred, ano_teste, sao_paulo):
        """Cria mapa comparando valores reais vs previstos"""
        print(f"\n🗺️ Criando mapas de regressão - {ano_teste}")

        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

        # Mapa 1: Valores Reais
        sao_paulo.boundary.plot(ax=ax1, color='black', linewidth=0.5, alpha=0.5)
        grid_real = grid_teste.copy()
        grid_real['valor_real'] = y_real
        grid_real['valor_real_log'] = np.log1p(grid_real['valor_real'])

        grid_real.plot(ax=ax1, column='valor_real_log', cmap='Reds',
                       legend=True, markersize=1,
                       legend_kwds={'label': 'Log(Contagem Real + 1)'})

        ax1.set_title(f'📊 VALORES REAIS - {ano_teste}', fontsize=12, weight='bold')
        ax1.set_axis_off()

        # Mapa 2: Valores Preditos
        sao_paulo.boundary.plot(ax=ax2, color='black', linewidth=0.5, alpha=0.5)
        grid_pred = grid_teste.copy()
        grid_pred['valor_predito'] = y_pred
        grid_pred['valor_predito_log'] = np.log1p(grid_pred['valor_predito'])

        grid_pred.plot(ax=ax2, column='valor_predito_log', cmap='Reds',
                       legend=True, markersize=1,
                       legend_kwds={'label': 'Log(Contagem Predita + 1)'})

        ax2.set_title(f'🤖 VALORES PREDITOS - {ano_teste}', fontsize=12, weight='bold')
        ax2.set_axis_off()

        # Mapa 3: Erros (Resíduos)
        sao_paulo.boundary.plot(ax=ax3, color='black', linewidth=0.5, alpha=0.5)
        grid_erros = grid_teste.copy()
        grid_erros['erro'] = y_real - y_pred
        grid_erros['erro_abs'] = np.abs(grid_erros['erro'])

        grid_erros.plot(ax=ax3, column='erro_abs', cmap='RdBu_r',
                        legend=True, markersize=1,
                        legend_kwds={'label': 'Erro Absoluto'})

        mae = mean_absolute_error(y_real, y_pred)
        ax3.set_title(f'📈 ERROS DA PREVISÃO - {ano_teste}\nMAE: {mae:.2f}',
                      fontsize=12, weight='bold')
        ax3.set_axis_off()

        plt.tight_layout()
        plt.savefig(f'mapa_regressao_{ano_teste}.png', dpi=300, bbox_inches='tight')
        plt.show()

    def criar_grafico_dispersao(self, y_real, y_pred, ano_teste):
        """Cria gráfico de dispersão entre valores reais e previstos"""
        plt.figure(figsize=(10, 8))

        plt.scatter(y_real, y_pred, alpha=0.6, s=10)

        max_val = max(y_real.max(), y_pred.max())
        plt.plot([0, max_val], [0, max_val], 'r--', alpha=0.8, label='Previsão Perfeita')

        plt.xlabel('Valores Reais')
        plt.ylabel('Valores Preditos')
        plt.title(f'Dispersão: Real vs Predito - {ano_teste}\nR² = {r2_score(y_real, y_pred):.3f}')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(f'dispersao_regressao_{ano_teste}.png', dpi=300, bbox_inches='tight')
        plt.show()

    def criar_mapa_hotspots_detalhado(self, grid_teste, y_real, y_pred, ano_teste, sao_paulo, threshold):
        """Cria mapa detalhado comparando hotspots reais vs previstos"""
        print(f"🗺️ Criando mapa detalhado de hotspots - {ano_teste}")

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

        # Mapa 1: Hotspots Reais
        sao_paulo.boundary.plot(ax=ax1, color='black', linewidth=0.5, alpha=0.5)
        grid_real = grid_teste.copy()
        grid_real['hotspot_real'] = (y_real > threshold).astype(int)

        # Plotar não-hotspots e hotspots
        nao_hotspots = grid_real[grid_real['hotspot_real'] == 0]
        hotspots = grid_real[grid_real['hotspot_real'] == 1]

        if not nao_hotspots.empty:
            nao_hotspots.plot(ax=ax1, color='lightblue', alpha=0.6, markersize=1, label='Não Hotspot')
        if not hotspots.empty:
            hotspots.plot(ax=ax1, color='red', alpha=0.8, markersize=2, label='Hotspot Real')

        ax1.set_title(
            f'🔥 HOTSPOTS REAIS - {ano_teste}\nTotal: {hotspots.shape[0]:,} ({hotspots.shape[0] / len(grid_real) * 100:.1f}%)',
            fontsize=12, weight='bold')
        ax1.legend()
        ax1.set_axis_off()

        # Mapa 2: Hotspots Previstos
        sao_paulo.boundary.plot(ax=ax2, color='black', linewidth=0.5, alpha=0.5)
        grid_pred = grid_teste.copy()
        grid_pred['hotspot_predito'] = (y_pred > threshold).astype(int)

        nao_hotspots_pred = grid_pred[grid_pred['hotspot_predito'] == 0]
        hotspots_pred = grid_pred[grid_pred['hotspot_predito'] == 1]

        if not nao_hotspots_pred.empty:
            nao_hotspots_pred.plot(ax=ax2, color='lightblue', alpha=0.6, markersize=1, label='Não Hotspot (Previsto)')
        if not hotspots_pred.empty:
            hotspots_pred.plot(ax=ax2, color='red', alpha=0.8, markersize=2, label='Hotspot Previsto')

        ax2.set_title(
            f'🤖 HOTSPOTS PREVISTOS - {ano_teste}\nTotal: {hotspots_pred.shape[0]:,} ({hotspots_pred.shape[0] / len(grid_pred) * 100:.1f}%)',
            fontsize=12, weight='bold')
        ax2.legend()
        ax2.set_axis_off()

        plt.tight_layout()
        plt.savefig(f'mapa_hotspots_detalhado_{ano_teste}.png', dpi=300, bbox_inches='tight')
        plt.show()


# ============ MODELO PREDITIVO CORRETO ============
class CorrectPredictiveModel:
    """Modelo metodologicamente correto para predição temporal"""

    def __init__(self, config):
        self.config = config
        self.model = None
        self.processor = SafeTemporalProcessor(config)
        self.visualizator = RegressionVisualizator(config)  # ✅ VISUALIZAÇÃO INCLUÍDA

    def executar_predicao_temporal_correta(self):
        """Executa predição temporal CORRETA: passado → futuro"""
        print("🚀 INICIANDO PREDIÇÃO TEMPORAL CORRETA")
        print("=" * 60)
        print(f"🎯 CONFIGURAÇÃO:")
        print(f"   • Treino: {self.config.ANOS_TREINO}")
        print(f"   • Teste:  {self.config.ANO_TESTE}")
        print(f"   • Garantia: SEM vazamento temporal")
        print("=" * 60)

        try:
            # 1. Processar TREINO (apenas anos passados)
            X_treino, y_treino, grid_treino = self.processor.processar_conjunto_treino(
                self.config.ANOS_TREINO
            )

            # 2. Treinar modelo APENAS com dados de treino
            print("🎯 Treinando modelo XGBoost...")
            self.model = xgb.XGBRegressor(**self.config.XGB_PARAMS)
            self.model.fit(X_treino, y_treino)

            # 3. Processar TESTE (usando apenas informações do treino)
            X_teste, y_teste, grid_teste = self.processor.processar_conjunto_teste(
                self.config.ANO_TESTE
            )

            # 4. Fazer previsões para o FUTURO
            y_pred = self.model.predict(X_teste)

            # 5. ✅ GERAR VISUALIZAÇÕES
            print("🎨 Gerando visualizações...")
            sao_paulo = SpatialProcessor(self.config).carregar_limites_sp()

            # Criar mapas principais
            self.visualizator.criar_mapa_regressao(grid_teste, y_teste, y_pred, self.config.ANO_TESTE, sao_paulo)
            self.visualizator.criar_grafico_dispersao(y_teste, y_pred, self.config.ANO_TESTE)

            # 6. Avaliação rigorosa
            resultado = self._avaliar_predicao_rigorosa(y_teste, y_pred, grid_teste)

            # 7. Criar mapa de hotspots com threshold correto
            threshold = resultado['hotspots']['threshold']
            self.visualizator.criar_mapa_hotspots_detalhado(grid_teste, y_teste, y_pred, self.config.ANO_TESTE,
                                                            sao_paulo, threshold)

            # 8. Análise de importância
            self._analisar_importancia_features()

            return resultado

        except Exception as e:
            print(f"❌ Erro na predição: {e}")
            import traceback
            traceback.print_exc()
            return None

    def _avaliar_predicao_rigorosa(self, y_teste, y_pred, grid_teste):
        """Avaliação metodologicamente rigorosa"""
        print("\n📊 AVALIAÇÃO RIGOROSA DA PREDIÇÃO")
        print("=" * 50)

        # Métricas básicas
        mae = mean_absolute_error(y_teste, y_pred)
        rmse = np.sqrt(mean_squared_error(y_teste, y_pred))
        r2 = r2_score(y_teste, y_pred)

        print(f"📈 Métricas de Regressão:")
        print(f"   • R²:   {r2:.4f}")
        print(f"   • MAE:  {mae:.4f}")
        print(f"   • RMSE: {rmse:.4f}")

        # Análise de significância
        self._analisar_significancia_r2(r2, len(y_teste))

        # Converter para hotspots COM THRESHOLD CORRETO
        resultado_hotspots = self._avaliar_hotspots(y_teste, y_pred)

        # Análise de resíduos
        residuos = y_teste - y_pred
        print(f"\n🔍 Análise de Resíduos:")
        print(f"   • Média: {residuos.mean():.4f}")
        print(f"   • Std:   {residuos.std():.4f}")
        print(f"   • RMSE/Std(y): {rmse / y_teste.std():.4f}")

        return {
            'metricas_regressao': {'r2': r2, 'mae': mae, 'rmse': rmse},
            'hotspots': resultado_hotspots,
            'n_amostras': len(y_teste),
            'y_real_stats': {
                'mean': y_teste.mean(),
                'std': y_teste.std(),
                'min': y_teste.min(),
                'max': y_teste.max()
            }
        }

    def _analisar_significancia_r2(self, r2, n_amostras):
        """Analisa se o R² é estatisticamente significativo"""
        print(f"\n📊 Significância Estatística (R² = {r2:.4f}):")

        if r2 > 0.7:
            print("   ✅ EXCELENTE - Modelo explica a maioria da variância")
        elif r2 > 0.5:
            print("   📊 BOM - Modelo explica parte substancial da variância")
        elif r2 > 0.3:
            print("   ⚠️  MODERADO - Modelo tem algum poder explicativo")
        elif r2 > 0.1:
            print("   🚨 FRACO - Poder preditivo limitado")
        else:
            print("   ❌ MUITO FRACO - Modelo praticamente não explica a variância")

        # Regra prática: R² > 1/sqrt(n) para ser não-trivial
        limiar_nao_trivial = 1 / np.sqrt(n_amostras)
        if r2 > limiar_nao_trivial:
            print(f"   ✅ R² acima do limiar de não-trivialidade ({limiar_nao_trivial:.4f})")
        else:
            print(f"   ⚠️  R² abaixo do limiar de não-trivialidade ({limiar_nao_trivial:.4f})")

    def _avaliar_hotspots(self, y_real, y_pred, percentil=85):
        """Avaliação focada em hotspots COM THRESHOLD CORRETO"""
        # Usar percentil dos dados REAIS para definir threshold
        threshold = max(np.percentile(y_real, percentil), self.config.THRESHOLD_MINIMO)

        y_real_bin = (y_real > threshold).astype(int)
        y_pred_bin = (y_pred > threshold).astype(int)

        accuracy = accuracy_score(y_real_bin, y_pred_bin)
        precision = precision_score(y_real_bin, y_pred_bin, zero_division=0)
        recall = recall_score(y_real_bin, y_pred_bin, zero_division=0)
        f1 = f1_score(y_real_bin, y_pred_bin, zero_division=0)

        print(f"\n🔥 AVALIAÇÃO DE HOTSPOTS (threshold: {threshold:.2f} crimes):")
        print(f"   • Hotspots reais: {y_real_bin.sum():,} ({y_real_bin.sum() / len(y_real) * 100:.1f}%)")
        print(f"   • Hotspots previstos: {y_pred_bin.sum():,} ({y_pred_bin.sum() / len(y_pred) * 100:.1f}%)")
        print(f"   • Acurácia:  {accuracy:.4f}")
        print(f"   • Precisão:  {precision:.4f}")
        print(f"   • Recall:    {recall:.4f}")
        print(f"   • F1-Score:  {f1:.4f}")

        # Diagnóstico adicional
        if y_pred_bin.sum() > len(y_pred) * 0.5:
            print("   ⚠️  ALERTA: Mais de 50% das células classificadas como hotspots!")
        if threshold == self.config.THRESHOLD_MINIMO:
            print("   ⚠️  ALERTA: Threshold atingiu o valor mínimo")

        return {
            'threshold': threshold,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'n_hotspots_real': y_real_bin.sum(),
            'n_hotspots_pred': y_pred_bin.sum()
        }

    def _analisar_importancia_features(self):
        """Análise rigorosa da importância das features"""
        if self.model is None:
            return

        importancia = self.model.feature_importances_
        features = self.processor.features_para_usar

        df_importancia = pd.DataFrame({
            'feature': features,
            'importance': importancia
        }).sort_values('importance', ascending=False)

        print(f"\n🔍 IMPORTÂNCIA DAS FEATURES:")
        for _, row in df_importancia.iterrows():
            print(f"   • {row['feature']}: {row['importance']:.4f}")

        # Análise da feature mais importante
        top_feature = df_importancia.iloc[0]
        print(f"   🎯 Feature mais importante: {top_feature['feature']} ({top_feature['importance']:.4f})")


# ============ VALIDAÇÃO CRUZADA TEMPORAL ============
class CorrectTemporalValidator:
    """Validação cruzada temporal CORRETA"""

    def __init__(self, config):
        self.config = config

    def validar_abordagem_temporal(self):
        """Valida a abordagem temporal de forma correta"""
        print("\n🔄 VALIDAÇÃO DA ABORDAGEM TEMPORAL")
        print("=" * 50)

        anos_ordenados = sorted(self.config.ANOS_DISPONIVEIS)
        resultados = []

        # Walk-forward validation
        for i in range(1, len(anos_ordenados)):
            anos_treino = anos_ordenados[:i]
            ano_teste = anos_ordenados[i]

            # Pular se ano_teste for o ano de teste principal
            if ano_teste == self.config.ANO_TESTE:
                continue

            print(f"   🔁 {anos_treino} → {ano_teste}")

            # Configuração temporária
            class TempConfig:
                def __init__(self, anos_treino, ano_teste, anos_disponiveis, base_config):
                    self.ANOS_TREINO = anos_treino
                    self.ANO_TESTE = ano_teste
                    self.ANOS_DISPONIVEIS = anos_disponiveis
                    self.XGB_PARAMS = base_config.XGB_PARAMS
                    self.DATA_PATH = base_config.DATA_PATH
                    self.LAT_COL = base_config.LAT_COL
                    self.LON_COL = base_config.LON_COL
                    self.CELL_SIZE = base_config.CELL_SIZE
                    self.HOTSPOT_PERCENTIL = base_config.HOTSPOT_PERCENTIL
                    self.THRESHOLD_MINIMO = base_config.THRESHOLD_MINIMO

            config_temp = TempConfig(anos_treino, ano_teste, anos_ordenados[:i + 1], self.config)

            # Executar predição
            model = CorrectPredictiveModel(config_temp)
            resultado = model.executar_predicao_temporal_correta()

            if resultado:
                resultados.append({
                    'anos_treino': anos_treino,
                    'ano_teste': ano_teste,
                    'r2': resultado['metricas_regressao']['r2']
                })

        # Resumo da validação
        if resultados:
            r2_values = [r['r2'] for r in resultados]
            print(f"\n📊 RESUMO VALIDAÇÃO CRUZADA:")
            print(f"   • R² médio: {np.mean(r2_values):.4f}")
            print(f"   • Std R²:   {np.std(r2_values):.4f}")
            print(f"   • Min R²:   {np.min(r2_values):.4f}")
            print(f"   • Max R²:   {np.max(r2_values):.4f}")

        return resultados


# ============ EXECUÇÃO PRINCIPAL ============
def main_correta():
    """Execução metodologicamente correta"""
    start_time = time.time()

    config = Config()

    print("🎯 PREDIÇÃO TEMPORAL - VERSÃO CORRIGIDA")
    print("=" * 60)
    print("GARANTIAS:")
    print("   • SEM vazamento temporal")
    print("   • Treino: apenas anos passados")
    print("   • Teste: apenas ano futuro")
    print("   • Processamento separado")
    print("=" * 60)

    # Predição principal
    model = CorrectPredictiveModel(config)
    resultado_principal = model.executar_predicao_temporal_correta()

    # Validação temporal
    validator = CorrectTemporalValidator(config)
    resultados_validacao = validator.validar_abordagem_temporal()

    tempo_total = time.time() - start_time
    print(f"\n⏱️  Tempo total: {tempo_total / 60:.2f} minutos")

    if resultado_principal:
        r2 = resultado_principal['metricas_regressao']['r2']
        if r2 > 0.3:
            print("✅ PREDIÇÃO BEM-SUCEDIDA - Modelo tem poder preditivo")
        else:
            print("⚠️  PREDIÇÃO FRACA - Modelo tem poder preditivo limitado")

    return resultado_principal, resultados_validacao


if __name__ == "__main__":
    try:
        import xgboost as xgb

        print(f"✅ XGBoost disponível - versão {xgb.__version__}")
        resultado_principal, validacao = main_correta()
    except ImportError:
        print("❌ XGBoost não instalado. Execute: pip install xgboost")
